In [1]:
import tkinter as tk
from tkinter import scrolledtext
import socket
import threading


In [2]:
class ClientGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Chat Client")

        self.text_area = scrolledtext.ScrolledText(self.root, width=40, height=10)
        self.text_area.grid(row=0, column=0, padx=10, pady=10)

        self.entry = tk.Entry(self.root, width=30)
        self.entry.grid(row=1, column=0, padx=10, pady=5)

        self.send_button = tk.Button(self.root, text="Send", command=self.send_message)
        self.send_button.grid(row=1, column=1, padx=5, pady=5)

        self.quit_button = tk.Button(self.root, text="Quit", command=self.quit)
        self.quit_button.grid(row=1, column=2, padx=5, pady=5)

        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect(('localhost', 9999))

        receive_thread = threading.Thread(target=self.receive_messages)
        receive_thread.start()

    def send_message(self):
        message = self.entry.get()
        self.text_area.insert(tk.END, f"Me: {message}\n")
        self.client_socket.send(message.encode('utf-8'))
        self.entry.delete(0, tk.END)

    def receive_messages(self):
        while True:
            try:
                message = self.client_socket.recv(1024).decode('utf-8')
                if not message:  # If server closes the connection
                    break
                self.text_area.insert(tk.END, f"Server: {message}\n")
            except ConnectionAbortedError:
                break

        # If the loop breaks, it means the connection is closed
        self.client_socket.close()
        self.root.destroy()

    def quit(self):
        self.client_socket.close()  # Close the socket
        self.root.destroy()

In [3]:

if __name__ == "__main__":
    root = tk.Tk()
    app = ClientGUI(root)
    root.mainloop()